<a href="https://colab.research.google.com/github/anjalikm05/DataAnalytics/blob/master/ExtractiveSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install --quiet transformers[sentencepiece]

In [28]:
f = open('/content/sample.txt','r')
data = ' '.join(f.read().strip().split('\\n'))
data

'My dear countrymen, Namaskar. Generally, this period is full of festivals; fairs are held at various places; it’s the time for religious ceremonies too. During these times of Corona crises, on the one hand people are full of exaltation and enthusiasm too; and yet in a way there’s also a discipline that touches our heart. Broadly speaking in a way, there is a feeling of responsibility amongst citizens. People are getting along with their day to day tasks, while taking care of themselves and others as well. At every event being organised in the country, the kind of patience and simplicity being witnessed this time is unprecedented. Even Ganeshotsav is being celebrated online at certain places; at most places eco-friendly Ganesh idols have been installed. Friends, if we observe very minutely, one thing will certainly draw our attention- our festivals and the environment. There has always been a deep connect between the two. On the one hand, our festivals implicitly convey the message of 

In [29]:
len(data)

21649

In [30]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [31]:
tokenizer.model_max_length

1024

In [32]:
tokenizer.max_len_single_sentence

1022

In [33]:
tokenizer.num_special_tokens_to_add()

2

In [34]:
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

98

In [36]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

5

In [37]:
[len(tokenizer.tokenize(c)) for c in chunks]

[1014, 1017, 1004, 1012, 585]

In [38]:
[len(tokenizer(c).input_ids) for c in chunks]

[1016, 1019, 1006, 1014, 587]

In [39]:
sum([len(tokenizer(c).input_ids) for c in chunks])

4642

In [40]:
len(tokenizer(data).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (4633 > 1024). Running this sequence through the model will result in indexing errors


4633

In [41]:
sum([len(tokenizer.tokenize(c)) for c in chunks])

4632

In [42]:
len(tokenizer.tokenize(data))

4631

In [43]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [44]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


 During Corona crises, people are full of exaltation and enthusiasm, yet in a way there’s also a discipline that touches our heart. People are getting along with their day to day tasks, while taking care of themselves and others as well. Onam is a festival linked with our agriculture and is also a time of a new beginning for our rural economy.
 The Global Toy Industry is of more than 7 lakh crore rupees but, India’s share is very little in this. Some parts of India are developing also as Toy clusters, that is, as centres of toys. We can do two things through toys – bring back the glorious past in our lives and also spruce up our golden future.
 At the beginning of this month an app innovation challenge was put before the youth of the country. The month of September will be observed as Nutrition Month in the entire nation. Nutrition and proper nourishment as well play a very big role in the nation's development. The better nutrition a child imbibes in the womb, greater is the mental dev